# Note :    Riêng **Colab** phải **Ctrl+S** hoặc **Save** thì mới update được nha mọi người.

In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import pyplot
# from sklearn.model_selection import train_test_split


# 1.Data Preparation

## 1.1. Data Shuffling

### 1.1.1 Daily Activity


In [71]:
# Correct URL to the raw CSV file on GitHub
dailyActivity = pd.read_csv('../data/dailyActivity_merged.csv')
dailyActivity.head()

,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
0,1503960366,3/25/2016,11004,7.11,7.11,0.0,2.57,0.46,4.07,0.0,33,12,205,804,1819
1,1503960366,3/26/2016,17609,11.55,11.55,0.0,6.92,0.73,3.91,0.0,89,17,274,588,2154
2,1503960366,3/27/2016,12736,8.53,8.53,0.0,4.66,0.16,3.71,0.0,56,5,268,605,1944
3,1503960366,3/28/2016,13231,8.93,8.93,0.0,3.19,0.79,4.95,0.0,39,20,224,1080,1932
4,1503960366,3/29/2016,12041,7.85,7.85,0.0,2.16,1.09,4.61,0.0,28,28,243,763,1886


In [72]:
dailyActivity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 457 entries, 0 to 456
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Id                        457 non-null    int64  
 1   ActivityDate              457 non-null    object 
 2   TotalSteps                457 non-null    int64  
 3   TotalDistance             457 non-null    float64
 4   TrackerDistance           457 non-null    float64
 5   LoggedActivitiesDistance  457 non-null    float64
 6   VeryActiveDistance        457 non-null    float64
 7   ModeratelyActiveDistance  457 non-null    float64
 8   LightActiveDistance       457 non-null    float64
 9   SedentaryActiveDistance   457 non-null    float64
 10  VeryActiveMinutes         457 non-null    int64  
 11  FairlyActiveMinutes       457 non-null    int64  
 12  LightlyActiveMinutes      457 non-null    int64  
 13  SedentaryMinutes          457 non-null    int64  
 14  Calories  

In [73]:
from sklearn.utils import shuffle

dailyActivity = shuffle(dailyActivity)
dailyActivity.head()

,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
71,1927972279,4/12/2016,24,0.02,0.02,0.000000,0.00,0.00,0.02,0.00,0,0,3,161,942
284,4702921684,4/6/2016,7605,6.18,6.18,0.000000,0.33,0.21,5.63,0.00,4,4,248,708,2943
215,4057192912,3/30/2016,5419,4.00,4.57,0.055843,0.00,0.00,3.91,0.06,0,0,145,1295,2259
52,1844505072,4/5/2016,0,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0,0,0,1440,1347
240,4319703577,4/9/2016,0,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0,0,0,1440,0


### 1.1.2. Heart Rate Second

In [74]:
# Correct URL to the raw CSV file on GitHub
heartRate = pd.read_csv('../data/heartrate_seconds_merged.csv')
heartRate.head()

,Id,Time,Value
0,2022484408,4/12/2016 7:21:00 AM,97
1,2022484408,4/12/2016 7:21:05 AM,102
2,2022484408,4/12/2016 7:21:10 AM,105
3,2022484408,4/12/2016 7:21:20 AM,103
4,2022484408,4/12/2016 7:21:25 AM,101


In [75]:
heartRate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2483658 entries, 0 to 2483657
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   Id      int64 
 1   Time    object
 2   Value   int64 
dtypes: int64(2), object(1)
memory usage: 56.8+ MB


In [76]:
from sklearn.utils import shuffle

heartRate = shuffle(heartRate)
heartRate.head()

,Id,Time,Value
1475420,5577150313,4/30/2016 7:54:35 AM,54
537480,4020332650,5/8/2016 9:03:53 PM,75
990010,4558609924,5/4/2016 8:49:35 PM,70
1659630,6117666160,5/2/2016 6:18:45 AM,95
1038935,5553957443,4/12/2016 7:56:35 AM,86


### 1.1.3. Hourly Calories

In [77]:
# Correct URL to the raw CSV file on GitHub
hourlyCalories = pd.read_csv('../data/hourlyCalories_merged.csv')
hourlyCalories.head()

,Id,ActivityHour,Calories
0,1503960366,3/12/2016 12:00:00 AM,48
1,1503960366,3/12/2016 1:00:00 AM,48
2,1503960366,3/12/2016 2:00:00 AM,48
3,1503960366,3/12/2016 3:00:00 AM,48
4,1503960366,3/12/2016 4:00:00 AM,48


In [78]:
hourlyCalories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24084 entries, 0 to 24083
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Id            24084 non-null  int64 
 1   ActivityHour  24084 non-null  object
 2   Calories      24084 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 564.6+ KB


In [79]:
from sklearn.utils import shuffle

hourlyCalories = shuffle(hourlyCalories)
hourlyCalories.head()

,Id,ActivityHour,Calories
7962,3372868164,4/1/2016 10:00:00 AM,101
5316,2320127002,3/16/2016 6:00:00 AM,55
5120,2026352035,4/8/2016 12:00:00 PM,58
10487,4319703577,3/14/2016 10:00:00 AM,125
2250,1844505072,3/14/2016 3:00:00 AM,56


### 1.1.4. Hourly Intensities

In [80]:
# Correct URL to the raw CSV file on GitHub
hourlyIntensities = pd.read_csv('../data/hourlyCalories_merged.csv')
hourlyIntensities.head()

,Id,ActivityHour,Calories
0,1503960366,3/12/2016 12:00:00 AM,48
1,1503960366,3/12/2016 1:00:00 AM,48
2,1503960366,3/12/2016 2:00:00 AM,48
3,1503960366,3/12/2016 3:00:00 AM,48
4,1503960366,3/12/2016 4:00:00 AM,48


In [81]:
hourlyIntensities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24084 entries, 0 to 24083
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Id            24084 non-null  int64 
 1   ActivityHour  24084 non-null  object
 2   Calories      24084 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 564.6+ KB


In [82]:
from sklearn.utils import shuffle

hourlyIntensities = shuffle(hourlyIntensities)
hourlyIntensities.head()

,Id,ActivityHour,Calories
8230,3977333714,3/13/2016 11:00:00 PM,49
21970,8583815059,3/14/2016 11:00:00 AM,111
11074,4319703577,4/7/2016 9:00:00 PM,75
7253,2873212765,4/3/2016 5:00:00 AM,52
19961,8053475328,3/24/2016 10:00:00 AM,73


### 1.1.5. Sleep Day

In [83]:
# Correct URL to the raw CSV file on GitHub
sleepDay = pd.read_csv('../data/sleepDay_merged.csv')
sleepDay.head()

,Id,SleepDay,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
0,1503960366,4/12/2016 12:00:00 AM,1,327,346
1,1503960366,4/13/2016 12:00:00 AM,2,384,407
2,1503960366,4/15/2016 12:00:00 AM,1,412,442
3,1503960366,4/16/2016 12:00:00 AM,2,340,367
4,1503960366,4/17/2016 12:00:00 AM,1,700,712


In [84]:
sleepDay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413 entries, 0 to 412
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Id                  413 non-null    int64 
 1   SleepDay            413 non-null    object
 2   TotalSleepRecords   413 non-null    int64 
 3   TotalMinutesAsleep  413 non-null    int64 
 4   TotalTimeInBed      413 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 16.3+ KB


In [85]:
from sklearn.utils import shuffle

sleepDay = shuffle(sleepDay)
sleepDay.head()

,Id,SleepDay,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
411,8792009665,5/3/2016 12:00:00 AM,1,516,545
136,4319703577,5/6/2016 12:00:00 AM,1,450,491
39,2026352035,4/14/2016 12:00:00 AM,1,545,568
143,4388161847,4/15/2016 12:00:00 AM,1,499,526
304,6775888955,4/14/2016 12:00:00 AM,1,423,441


### 1.1.6. Weight Info

In [86]:
# Correct URL to the raw CSV file on GitHub
weightLogInfo = pd.read_csv('../data/weightLogInfo_merged.csv')
weightLogInfo.head()

,Id,Date,WeightKg,WeightPounds,Fat,BMI,IsManualReport,LogId
0,1503960366,5/2/2016 11:59:59 PM,52.599998,115.963147,22.0,22.650000,True,1462233599000
1,1503960366,5/3/2016 11:59:59 PM,52.599998,115.963147,NaN,22.650000,True,1462319999000
2,1927972279,4/13/2016 1:08:52 AM,133.500000,294.317120,NaN,47.540001,False,1460509732000
3,2873212765,4/21/2016 11:59:59 PM,56.700001,125.002104,NaN,21.450001,True,1461283199000
4,2873212765,5/12/2016 11:59:59 PM,57.299999,126.324875,NaN,21.690001,True,1463097599000


In [87]:
weightLogInfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Id              67 non-null     int64  
 1   Date            67 non-null     object 
 2   WeightKg        67 non-null     float64
 3   WeightPounds    67 non-null     float64
 4   Fat             2 non-null      float64
 5   BMI             67 non-null     float64
 6   IsManualReport  67 non-null     bool   
 7   LogId           67 non-null     int64  
dtypes: bool(1), float64(4), int64(2), object(1)
memory usage: 3.9+ KB


In [88]:
from sklearn.utils import shuffle

weightLogInfo = shuffle(weightLogInfo)
weightLogInfo.head()

,Id,Date,WeightKg,WeightPounds,Fat,BMI,IsManualReport,LogId
30,6962181067,4/30/2016 11:59:59 PM,61.000000,134.481980,NaN,23.820000,True,1462060799000
26,6962181067,4/25/2016 11:59:59 PM,61.700001,136.025217,NaN,24.100000,True,1461628799000
65,8877689391,5/11/2016 6:51:47 AM,85.400002,188.274775,NaN,25.559999,False,1462949507000
63,8877689391,5/8/2016 7:35:53 AM,85.400002,188.274775,NaN,25.559999,False,1462692953000
57,8877689391,4/29/2016 6:49:55 AM,84.900002,187.172464,NaN,25.410000,False,1461912595000


## 1.2. Data Preprocessing

### 1.2.1. Cleaning the datasets

#### 1.2.1.a. Convert all the date columns into datetime object and rename it as ActivityDate

In [89]:
dailyActivity['ActivityDate'] = pd.to_datetime(dailyActivity['ActivityDate'])

In [90]:
def process_dataframe(df):
    # converting date column to datetime
    date_col = df.columns[1]
    df[date_col] = pd.to_datetime(df[date_col])

    # Creating separate 'date' and 'time' columns
    df['ActivityDate'] = df[date_col].dt.date
    df['ActivityTime'] = df[date_col].dt.strftime('%I:%M:%S %p')
    df['ActivityDate'] = pd.to_datetime(df['ActivityDate'])
    df.drop(columns=[date_col],inplace=True)

    return df

dataframes = [sleepDay, hourlyCalories, hourlyIntensities, heartRate, weightLogInfo]

for i, df in enumerate(dataframes):
    dataframes[i] = process_dataframe(df)

for i, df in enumerate(dataframes, 1):
    print(f"DataFrame {i}:\n", df.head())
    print(df['ActivityDate'].dtypes)

C:\Users\Admin\AppData\Local\Temp\ipykernel_7244\416243218.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[date_col] = pd.to_datetime(df[date_col])
C:\Users\Admin\AppData\Local\Temp\ipykernel_7244\416243218.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[date_col] = pd.to_datetime(df[date_col])


DataFrame 1:
              Id  TotalSleepRecords  TotalMinutesAsleep  TotalTimeInBed  \
411  8792009665                  1                 516             545   
136  4319703577                  1                 450             491   
39   2026352035                  1                 545             568   
143  4388161847                  1                 499             526   
304  6775888955                  1                 423             441   

    ActivityDate ActivityTime  
411   2016-05-03  12:00:00 AM  
136   2016-05-06  12:00:00 AM  
39    2016-04-14  12:00:00 AM  
143   2016-04-15  12:00:00 AM  
304   2016-04-14  12:00:00 AM  
datetime64[ns]
DataFrame 2:
                Id  Calories ActivityDate ActivityTime
7962   3372868164       101   2016-04-01  10:00:00 AM
5316   2320127002        55   2016-03-16  06:00:00 AM
5120   2026352035        58   2016-04-08  12:00:00 PM
10487  4319703577       125   2016-03-14  10:00:00 AM
2250   1844505072        56   2016-03-14  03:00:00

C:\Users\Admin\AppData\Local\Temp\ipykernel_7244\416243218.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[date_col] = pd.to_datetime(df[date_col])


#### 1.2.1.b.  Check if there are any duplicate records

In [91]:
grouped = dailyActivity.groupby(['Id', 'ActivityDate']).size().reset_index(name='Count')
duplicates = grouped[grouped['Count'] > 1]
print("\n Duplicates in activity record:")
print(duplicates)

grouped1 = sleepDay.groupby(['Id', 'ActivityDate','ActivityTime','TotalSleepRecords']).size().reset_index(name='Count')
duplicates1 = grouped1[grouped1['Count'] > 1]
print("\nDuplicates in sleep record:")
print(duplicates1)

grouped2 = hourlyCalories.groupby(['Id', 'ActivityDate','ActivityTime']).size().reset_index(name='Count')
duplicates2 = grouped2[grouped2['Count'] > 1]
print("\nDuplicates in hourly calories record:")
print(duplicates2)


grouped3 = hourlyIntensities.groupby(['Id', 'ActivityDate','ActivityTime']).size().reset_index(name='Count')
duplicates3 = grouped3[grouped3['Count'] > 1]
print("\nDuplicates in hourly intensities record:")
print(duplicates3)

grouped4 = heartRate.groupby(['Id', 'ActivityDate','ActivityTime']).size().reset_index(name='Count')
duplicates4 = grouped4[grouped4['Count'] > 1]
print("\nDuplicates in heartrate seconds record:")
print(duplicates4)


 Duplicates in activity record:
Empty DataFrame
Columns: [Id, ActivityDate, Count]
Index: []

Duplicates in sleep record:
             Id ActivityDate ActivityTime  TotalSleepRecords  Count
160  4388161847   2016-05-05  12:00:00 AM                  1      2
221  4702921684   2016-05-07  12:00:00 AM                  1      2
377  8378563200   2016-04-25  12:00:00 AM                  1      2

Duplicates in hourly calories record:
Empty DataFrame
Columns: [Id, ActivityDate, ActivityTime, Count]
Index: []

Duplicates in hourly intensities record:
Empty DataFrame
Columns: [Id, ActivityDate, ActivityTime, Count]
Index: []

Duplicates in heartrate seconds record:
Empty DataFrame
Columns: [Id, ActivityDate, ActivityTime, Count]
Index: []


In [92]:
duplicate_records = sleepDay[sleepDay.set_index(['Id', 'ActivityDate', 'ActivityTime', 'TotalSleepRecords']).index.isin(duplicates1.set_index(['Id', 'ActivityDate', 'ActivityTime', 'TotalSleepRecords']).index)]
print("\nDuplicate records in the original DataFrame:")
print(duplicate_records)

# Removing the duplicate records from the original DataFrame
sleepDay.drop_duplicates(subset=['Id', 'ActivityDate', 'ActivityTime', 'TotalSleepRecords'], inplace=True)


Duplicate records in the original DataFrame:
             Id  TotalSleepRecords  TotalMinutesAsleep  TotalTimeInBed  \
380  8378563200                  1                 388             402   
379  8378563200                  1                 388             402   
161  4388161847                  1                 471             495   
160  4388161847                  1                 471             495   
222  4702921684                  1                 520             543   
223  4702921684                  1                 520             543   

    ActivityDate ActivityTime  
380   2016-04-25  12:00:00 AM  
379   2016-04-25  12:00:00 AM  
161   2016-05-05  12:00:00 AM  
160   2016-05-05  12:00:00 AM  
222   2016-05-07  12:00:00 AM  
223   2016-05-07  12:00:00 AM  


##### Only the SleepDay dataset has duplicates.

In [93]:
import pandas as pd
import numpy as np

# Function to check for missing values including NaN, None, and empty strings
def check_missing_values(df):
    return {
        'null_values': df.isnull().values.sum(),
        'empty_strings': (df == '').values.sum(),
        'na_values': df.isna().values.sum()
    }

# Check for missing values in each DataFrame
print("Missing values in dailyActivity:")
print(check_missing_values(dailyActivity))

print("\nMissing values in sleepDay:")
print(check_missing_values(sleepDay))

print("\nMissing values in hourlyCalories:")
print(check_missing_values(hourlyCalories))

print("\nMissing values in hourlyIntensities:")
print(check_missing_values(hourlyIntensities))

print("\nMissing values in heartRate:")
print(check_missing_values(heartRate))

Missing values in dailyActivity:
{'null_values': np.int64(0), 'empty_strings': np.int64(0), 'na_values': np.int64(0)}

Missing values in sleepDay:
{'null_values': np.int64(0), 'empty_strings': np.int64(0), 'na_values': np.int64(0)}

Missing values in hourlyCalories:
{'null_values': np.int64(0), 'empty_strings': np.int64(0), 'na_values': np.int64(0)}

Missing values in hourlyIntensities:
{'null_values': np.int64(0), 'empty_strings': np.int64(0), 'na_values': np.int64(0)}

Missing values in heartRate:
{'null_values': np.int64(0), 'empty_strings': np.int64(0), 'na_values': np.int64(0)}


#### 1.2.1.c. Clone into train file

In [97]:
import os

clone_directory = 'clone_data'
train_file_path1 = os.path.join(clone_directory, 'train_dailyActivity.csv')
train_file_path2 = os.path.join(clone_directory, 'train_heartRate.csv')
train_file_path3 = os.path.join(clone_directory, 'train_hourlyCalories.csv')
train_file_path4 = os.path.join(clone_directory, 'train_hourlyIntensities.csv')
train_file_path5 = os.path.join(clone_directory, 'train_sleepDay.csv')
train_file_path6 = os.path.join(clone_directory, 'train_weighInfo.csv')

test_file_path1 = os.path.join(clone_directory, 'test_dailyActivity.csv')
test_file_path2 = os.path.join(clone_directory, 'test_heartRate.csv')
test_file_path3 = os.path.join(clone_directory, 'test_hourlyCalories.csv')
test_file_path4 = os.path.join(clone_directory, 'test_hourlyIntensities.csv')
test_file_path5 = os.path.join(clone_directory, 'test_sleepDay.csv')
test_file_path6 = os.path.join(clone_directory, 'test_weighInfo.csv')

if not os.path.exists(clone_directory):
    os.makedirs(clone_directory)

dailyActivity.to_csv(train_file_path1)
heartRate.to_csv(train_file_path2)
hourlyCalories.to_csv(train_file_path3)
hourlyIntensities.to_csv(train_file_path4)
sleepDay.to_csv(train_file_path5)
weightLogInfo.to_csv(train_file_path6)

dailyActivity.to_csv(test_file_path1)
heartRate.to_csv(test_file_path2)
hourlyCalories.to_csv(test_file_path3)
hourlyIntensities.to_csv(test_file_path4)
sleepDay.to_csv(test_file_path5)
weightLogInfo.to_csv(test_file_path6)


### 1.2.2. Test

#### 1.2.2.a. 

#### 1.2.2.b.

#### 1.2.2.c.

# 2. Selection

## 2.1. Information Gain

## 2.2. Forward Selection

## 2.3. Chi_Square

# 3. Split and Prepare Data

# 4. Model
#### Decision Tree, Logistic Regression, Random Forest, KNN, Naive Bayes, SVM

# 5. Apply Model